In [40]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, precision_score, accuracy_score,f1_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from nltk.tokenize import RegexpTokenizer
from sklearn.pipeline import Pipeline
from nltk.corpus import names
from collections import defaultdict
import re
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier

%matplotlib inline

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#train test validation split

# X,y = df.lemmatized,df.target

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [3]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_val = pd.read_csv('val.csv')

In [4]:
df_test

,preprocessed_text,target
0,theyve sold cray realised spending two year nu...,4
1,thanks doc info turn leave colormap update fra...,5
2,manual transmission speed difficult engage gea...,7
3,recently compiled xrpl source using gcc sun ev...,5
4,kratz comment show otherwise bingo question gl...,16
...,...,...
4568,yeah dont use clutch time either ive done ford...,7
4569,hello trouble using tcp onpredir printer redir...,2
4570,supposed signify prefer companionship person e...,18
4571,greg flame definitely intended bill making fun...,13


In [5]:
df_list = [df_train,df_test,df_val]
for df in df_list:
    df.astype({'target': 'int32'}).dtypes

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10287 entries, 0 to 10286
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   preprocessed_text  10287 non-null  object
 1   target             10287 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 160.9+ KB


In [7]:
X_train, y_train = df_train['preprocessed_text'], df_train['target']
X_test, y_test = df_test['preprocessed_text'], df_test['target']
X_val, y_val = df_val['preprocessed_text'], df_val['target']

In [8]:
y_train

0         7
1        10
2         3
3        13
4        10
         ..
10282    15
10283     3
10284     1
10285    16
10286    10
Name: target, Length: 10287, dtype: int64

In [9]:
#pipeline for logistic regression

pipe_logreg = Pipeline([
     ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

parameters_logreg = {
    'tfidf__min_df': [0.001],
    'tfidf__max_df': [0.5,0.75,1],
    'tfidf__max_features': [None, 5000, 10000, 50000],
    'tfidf__ngram_range': [(1,1),(1, 2)],  # unigrams or bigrams
    "clf__C": [0.1,1,10],
    "clf__class_weight": ['balanced'],
    "clf__solver": ['newton-cg','lbfgs'],
    "clf__max_iter":[80,100],
    "clf__multi_class":['multinomial','ovr']
}

gs_logreg = GridSearchCV(estimator=pipe_logreg,
            param_grid=parameters_logreg,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_logreg.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_logreg.best_score_)

# Best params
print('\nBest params:\n', gs_logreg.best_params_)

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   41.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 13.2min
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 21.6min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 32.8min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed: 48.2min
[Parallel(n_jobs=-1)]: Done 2880 out of 2880 | elapsed: 59.0min finished


Best accuracy: 0.711

Best params:
 {'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__max_iter': 80, 'clf__multi_class': 'multinomial', 'clf__solver': 'lbfgs', 'tfidf__max_df': 0.5, 'tfidf__max_features': None, 'tfidf__min_df': 0.001, 'tfidf__ngram_range': (1, 2)}


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [10]:
#pipeline for decision tree

pipe_tree = Pipeline([
     ('tfidf', TfidfVectorizer()),
    ('clf', DecisionTreeClassifier())
])
parameters_tree = {
    'tfidf__min_df': [0.001],
#     'tfidf__max_df': [0.75],
    'tfidf__ngram_range': [(1, 1)],  # unigrams or bigrams
    "clf__criterion": ["gini"],
    "clf__max_depth":[250,300,500],
    "clf__min_samples_split" : [150,200,250]
}

gs_tree = GridSearchCV(estimator=pipe_tree,
            param_grid=parameters_tree,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_tree.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_tree.best_score_)

# Best params
print('\nBest params:\n', gs_tree.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   44.4s finished


Best accuracy: 0.492

Best params:
 {'clf__criterion': 'gini', 'clf__max_depth': 300, 'clf__min_samples_split': 150, 'tfidf__min_df': 0.001, 'tfidf__ngram_range': (1, 1)}


In [11]:
#pipeline for random forest

# pipe_rf = Pipeline([
#      ('tfidf', TfidfVectorizer()),
#     ('clf', RandomForestClassifier())
# ])
# parameters_rf = {
#     'tfidf__min_df': [3,5,10],
#     'tfidf__max_df': [0.5, 0.75, 1.0],
#     'tfidf__ngram_range': [(1, 1), (1, 2)],  # unigrams or bigrams
#     "clf__criterion": ["gini", "entropy"],
#     "clf__max_depth":[75,100,150],
#     "clf__min_samples_split" : [75,100,150]
# }

# gs_rf = GridSearchCV(estimator=pipe_rf,
#             param_grid=parameters_rf,
#             scoring='accuracy',
#             cv=KFold(5,shuffle=True,random_state=42), 
#             return_train_score = True, verbose=1,n_jobs=-1)

# # Fit using grid search
# best_model = gs_rf.fit(X_train, y_train)

# # Best accuracy
# print('Best accuracy: %.3f' % gs_rf.best_score_)

# # Best params
# print('\nBest params:\n', gs_rf.best_params_)

In [38]:
#pipeline for NaiveBayes

pipe_nb = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',   MultinomialNB()),
])

parameters_nb = {
    'vect__ngram_range': [(1,1),(1,2)],
    'vect__min_df': [0.001]
#     'tfidf__min_df': [0.001]
#     'tfidf__max_df': [0.5, 0.75, 1.0],
#     'tfidf__ngram_range': [(1, 1), (1, 2)]  # unigrams or bigrams

}

gs_nb = GridSearchCV(estimator=pipe_nb,
            param_grid=parameters_nb,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_nb.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_nb.best_score_)
#
# # # Best params
print('\nBest params:\n', gs_nb.best_params_)

# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(pipe_nb, X_train, y_train, cv=5, scoring='accuracy')

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    9.1s remaining:    6.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.1s finished


Best accuracy: 0.698

Best params:
 {'vect__min_df': 0.001, 'vect__ngram_range': (1, 1)}


In [37]:
#pipeline for svm
from sklearn.svm import SVC
pipe_svm = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',   SVC()),
])

parameters_svm = {
    'vect__min_df': [0.001],
#     'tfidf__max_df': [0.5, 0.75, 1.0],
    'vect__ngram_range': [(1, 2)],  # unigrams or bigrams
    "clf__C": [0.1,1,10],
    "clf__kernel":['linear'],
    "clf__class_weight" : ['balanced'],
    "clf__decision_function_shape":['ovo'],
    "clf__gamma":[1e-07,1e-06,1e-05]
}

gs_svm = GridSearchCV(estimator=pipe_svm,
            param_grid=parameters_svm,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_svm.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_svm.best_score_)

# Best params
print('\nBest params:\n', gs_svm.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 11.0min finished


Best accuracy: 0.694

Best params:
 {'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__decision_function_shape': 'ovo', 'clf__gamma': 1e-07, 'clf__kernel': 'linear', 'vect__min_df': 0.001, 'vect__ngram_range': (1, 2)}


In [15]:
#pipeline for random forest take 2

pipe_rf2 = Pipeline([
     ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier())
])
parameters_rf2 = {
    'tfidf__min_df': [0.001],
#     'tfidf__max_df': [1.0],
    'tfidf__ngram_range': [(1, 2)],  #bigrams
    "clf__criterion": ["gini"],
    "clf__max_depth":[500,1000],
    "clf__min_samples_split" : [20,30,40]
}

gs_rf2 = GridSearchCV(estimator=pipe_rf2,
            param_grid=parameters_rf2,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_rf2.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_rf2.best_score_)

# Best params
print('\nBest params:\n', gs_rf2.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.4min finished


Best accuracy: 0.646

Best params:
 {'clf__criterion': 'gini', 'clf__max_depth': 1000, 'clf__min_samples_split': 20, 'tfidf__min_df': 0.001, 'tfidf__ngram_range': (1, 2)}


In [16]:
#pipeline for knn

pipe_knn = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svd',   TruncatedSVD(algorithm='randomized')),
    ('clf',   KNeighborsClassifier()),
])

parameters_knn = {
    'tfidf__min_df': [0.001],
#     'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__ngram_range': [(1,1),(1, 2)],  #  bigrams
    "svd__n_components": [50,75,100],
    "clf__n_neighbors": [5,10,15],
    "clf__metric": ['minkowski']
}

gs_knn = GridSearchCV(estimator=pipe_knn,
            param_grid=parameters_knn,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_knn.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_knn.best_score_)

# Best params
print('\nBest params:\n', gs_knn.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  6.2min finished


Best accuracy: 0.530

Best params:
 {'clf__metric': 'minkowski', 'clf__n_neighbors': 15, 'svd__n_components': 75, 'tfidf__min_df': 0.001, 'tfidf__ngram_range': (1, 2)}


In [52]:
#pipeline for SGD

pipe_sgd = Pipeline([
    ('vect',CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',   SGDClassifier()),
])

parameters_sgd = {
    'vect__min_df': [0.001],
#     'tfidf__max_df': [0.5, 0.75, 1.0],
    'vect__ngram_range': [(1,1),(1, 2)],  #  bigrams
#     "clf__n_neighbors": [5,10,15],
#     "clf__metric": ['minkowski']
    "clf__penalty": ['l2','l1','elasticnet'],
    "clf__loss":['hinge','log','perceptron'],
    "clf__alpha":[1e-3,1e-2,1e-1]
}

gs_sgd = GridSearchCV(estimator=pipe_sgd,
            param_grid=parameters_sgd,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_sgd.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_sgd.best_score_)

# Best params
print('\nBest params:\n', gs_sgd.best_params_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  4.7min finished


Best accuracy: 0.703

Best params:
 {'clf__alpha': 0.001, 'clf__loss': 'hinge', 'clf__penalty': 'l2', 'vect__min_df': 0.001, 'vect__ngram_range': (1, 2)}


In [17]:
#pipeline for xgboost

# pipe_xgb = Pipeline([
#     ('tfidf', TfidfVectorizer(min_df=0.001)),
#     ('svd',   TruncatedSVD(algorithm='randomized')),
#     ('clf',   XGBClassifier(objective='multi:softmax', num_class=20)),
# ])

# parameters_xgb = {
# #     'tfidf__min_df': [0.001],
# #     'tfidf__max_df': [0.5, 0.75, 1.0],
#     'tfidf__ngram_range': [(1,2),(1, 2)],  #  bigrams
#     "svd__n_components": [500],
#     "clf__n_estimators": [500],
#     "clf__learning_rate":[0.01,0.1,1],
#     "clf__min_samples_split" : [50,100]
# }

# gs_xgb = GridSearchCV(estimator=pipe_xgb,
#             param_grid=parameters_xgb,
#             scoring='accuracy',
#             cv=KFold(5,shuffle=True,random_state=42), 
#             return_train_score = True, verbose=1,n_jobs=-1)

# # Fit using grid search
# best_model = gs_xgb.fit(X_train, y_train)

# # Best accuracy
# print('Best accuracy: %.3f' % gs_xgb.best_score_)

# # Best params
# print('\nBest params:\n', gs_xgb.best_params_)